In [1]:
import pandas as pd

## --------------------------------- TOP 200 KOSPI ---------------------------------

In [4]:
# top_200 기업리스트 불러오기 
file_path = 'top200_name_list.csv'
top_200 = pd.read_csv(file_path, sep='\t')

#### ------------------------ Green GR 인증 --------------------------

In [45]:
# 우수 재활용(GR) 인증업체 현황 (제품 수량x)
excel_file_path = './green_datas/green_company_GR.xlsx'
data = pd.read_excel(excel_file_path)

df_GR = pd.DataFrame(data.values[1:], columns=data.iloc[0])
df_GR = df_GR.drop(columns=['GR기준', '규격명', '종류','유효인증번호', '만료기간','인증일자', '대표자', '순번', '사업자등록번호', '소 재 지']) # 

# '~'를 기준으로 '시작'과 '종료' 분할
df_GR[['시작', '종료']] = df_GR['유효기간'].str.split('~', expand=True)
df_GR['시작'] = df_GR['시작'].str.strip().str.replace('.', '-')
df_GR['종료'] = df_GR['종료'].str.strip().str.replace('.', '-')

# '0000-00-00' 형식으로 날짜 변환
df_GR['시작'] = pd.to_datetime(df_GR['시작'].str.replace('\n', ''), errors='coerce').dt.strftime('%Y-%m-%d')
df_GR['종료'] = pd.to_datetime(df_GR['종료'], errors='coerce').dt.strftime('%Y-%m-%d')

df_GR = df_GR.drop(columns='유효기간')
df_GR = df_GR.rename(columns={'업체명': '기업명'})
df_GR['기업명'] = df_GR['기업명'].str.replace("\(주\)|주식회사|㈜|주\)|\(유\)|\)|에스아이씨엔티\n\(구.|\(합|\(구. ", "", regex=True)

# 시작일 연도만 남겨놓기
df_GR = df_GR.drop(columns='종료')
# df_GR['종류'] = df_GR['종류'].fillna('-', inplace=True)
df_GR = df_GR.dropna()

df_GR['시작'] = pd.to_datetime(df_GR['시작'])
df_GR['시작'] = df_GR['시작'].dt.year.astype(int) 
df_GR = df_GR.rename(columns={'시작': '인증연도'})
df_GR = df_GR[['기업명', '인증연도', '분야']]

# 기업명칭 통일
rename = {
    '한일시멘트 포항공장': '한일시멘트',
    '현대제철 인천공장': '현대제철',
    '현대제철 포항1공장': '현대제철',
    '현대제철 포항2공장': '현대제철',
    '포스코 포항제철소': '포스코인터내셔널',
}

for old_name, new_name in rename.items():
    df_GR['기업명'] = df_GR['기업명'].replace(old_name, new_name, regex=True)
    df_GR['기업명'] = df_GR['기업명'].str.strip()
rename_df_GR = df_GR
rename_df_GR

,기업명,인증연도,분야
0,대한제지,1997,폐지
1,전주페이퍼,1997,폐지
2,한일시멘트,1999,폐요업
3,한라시멘트 광양,1999,폐요업
4,와이에이치 충주공장,1999,폐플라스틱
...,...,...,...
361,중산현대,2024,폐요업
362,유모아아스콘,2024,폐요업
363,태인,2024,폐요업
364,성경,2024,폐요업


In [7]:
# # rename_df_GR, top_200 두 데이터의 공통된 단어의 기업명
# words_df1 = set(' '.join(rename_df_GR['기업명']).split())
# words_df2 = set(' '.join(top_200['기업명']).split())

# common_words = set()
# for word1 in words_df1:
#     for word2 in words_df2:
#         if word1 in word2 or word2 in word1:
#             common_words.add(word1)

# print("두 데이터프레임의 '기업명' 열에서 포함된 공통된 단어:", common_words)

두 데이터프레임의 '기업명' 열에서 포함된 공통된 단어: {'경기아스콘산업', '한일시멘트', '동서개발', '포스코인터내셔널', '현대제철', '제일'}


In [5]:
# GR인증 기업리스트 중 top 200의 인증 기록 / '현대제철 인천공장,포항1공장, 포항2공장': '현대제철' 기업명을 현대제철로 바꾸면서 값이 동일하게 보는 것임
GR_companies = ['한일시멘트', '현대제철', '포스코인터내셔널']

top200_GR = rename_df_GR[rename_df_GR['기업명'].isin(GR_companies)]
top200_GR = top200_GR.reset_index(drop=True)
top200_GR

,기업명,인증연도,분야
0,한일시멘트,1999,폐요업
1,포스코인터내셔널,2021,폐금속
2,포스코인터내셔널,2021,폐금속
3,현대제철,2022,폐금속
4,현대제철,2022,폐금속
5,현대제철,2022,폐금속
6,현대제철,2022,폐금속
7,현대제철,2022,폐금속
8,현대제철,2022,폐금속
9,현대제철,2022,폐금속


#### ------------------------ Green 저탄소 인증 --------------------------

In [6]:
# 저탄소 인증제품 현황_최근 자료
excel_file_path = './green_datas/green_company_저탄소_1.xlsx'
data = pd.read_excel(excel_file_path)
# 첫 번째 행을 칼럼으로 사용하여 데이터프레임을 생성
df_C_1 = pd.DataFrame(data.values[4:], columns=data.iloc[3])

In [7]:
# 저탄소 인증제품 현황_오래된 자료
excel_file_path = './green_datas/green_company_저탄소_2.xlsx'
data = pd.read_excel(excel_file_path)
# 첫 번째 행을 칼럼으로 사용하여 데이터프레임을 생성
df_C_2 = pd.DataFrame(data.values[4:], columns=data.iloc[3])

In [34]:
# 저탄소 인증업체 현황 (제품 수량x)
df_C = pd.concat([df_C_1, df_C_2])
df_C = df_C.drop_duplicates(subset=['인증번호'])

# '인증번호' 기준으로 나열
df_C['인증번호'] = df_C['인증번호'].str.extract('(\d+)').astype(int)
df_C = df_C.sort_values(by='인증번호')

df_C = df_C.drop(columns=['연번', '사업자등록번호', '영향범주', '본사 주소', '소분류(상세분류)' ])

# # '기업명' 카운트 후 중복되는 '기업명' 제거
# counts = df_c['기업명'].value_counts()
# df_c['인증제품'] = df_c['기업명'].map(counts)
df_C = df_C.drop_duplicates(subset=['기업명']) 

# '인증번호'의 년도로 시작일자 수정
df_C['시작'] = pd.to_datetime(df_C['시작'])
df_C['인증번호'] = df_C['인증번호'].astype(str) + '-' + df_C['시작'].dt.strftime('%m-%d')
df_C = df_C.drop(columns='시작')
df_C = df_C.rename(columns={'인증번호': '시작'})
df_C = df_C.reset_index(drop=True)
df_C['기업명'] = df_C['기업명'].str.replace("\(주\)|주식회사|㈜|주\)|\(유\)|\)|에스아이씨엔티\n\(구.", "", regex=True)

# 시작일 연도만 남겨놓기
df_C = df_C.drop(columns='종료')
df_C = df_C.dropna()

df_C['시작'] = pd.to_datetime(df_C['시작'])
df_C['시작'] = df_C['시작'].dt.year.astype(int) 
df_C = df_C.rename(columns={'시작': '인증연도'})
df_C = df_C.rename(columns={'인증제품명': '제품명'})

# 기업명칭 통일
rename = {
    '롯데칠성음료': '롯데칠성',
    '금호석유화학': '금호석유',
    '포스코': '포스코인터내셔널',
}

for old_name, new_name in rename.items():
    df_C['기업명'] = df_C['기업명'].replace(old_name, new_name, regex=True)
    df_C['기업명'] = df_C['기업명'].str.strip()
rename_df_C = df_C
rename_df_C 

3,기업명,인증연도,제품명
0,풀무원샘물,2013,풀무원샘물 [2L]
1,케이씨씨,2013,일반 석고보드 [9.5T]
2,케이씨씨실리콘,2015,SL1000(소시지형)
3,벽산,2016,석고 시멘트판(아스텍스) 6T
4,크나우프석고보드,2016,일반석고보드 [9.5T]
...,...,...,...
232,동진피씨이,2024,건축용PC-거더[40Mpa]
233,케이엑스인텍,2024,BID-AT200
234,영진레미콘,2024,"레디믹스트 콘크리트[25-21-150,수도권]"
235,용원레미콘,2024,"레디믹스트 콘크리트[25-30-150,경상도]"


In [20]:
# # '기업명' 열의 값을 단어로 분리하여 리스트에 저장
# words_df1 = set(' '.join(rename_df_C['기업명']).split())
# words_df2 = set(' '.join(top_200['기업명']).split())

# # 두 데이터프레임에서 발견된 단어의 교집합 찾기
# common_words = set()
# for word1 in words_df1:
#     for word2 in words_df2:
#         if word1 in word2 or word2 in word1:
#             common_words.add(word1)

# print("두 데이터프레임의 '기업명' 열에서 포함된 공통된 단어:", common_words)

두 데이터프레임의 '기업명' 열에서 포함된 공통된 단어: {'삼양식품', '영풍파일', '삼성', '롯데칠성', '금호석유', '포스코인터내셔널인터내셔널', '강원', '아이에스동서', '삼성전자', '한일시멘트', '동원', '현대제철'}


In [10]:
# 저탄소인증 기업리스트 중 top 200의 인증 기록
C_companies = ['삼성전자', '금호석유', '한일시멘트','롯데칠성','아이에스동서', '현대제철','삼양식품', '포스코인터내셔널']

top200_C = rename_df_C[rename_df_C['기업명'].isin(C_companies)]
top200_C = top200_C.reset_index(drop=True)
top200_C

3,기업명,인증연도,제품명
0,포스코인터내셔널,2019,열연강판(구조용)
1,한일시멘트,2020,레디믹스트 콘크리트[25-24-120]
2,삼성전자,2020,포터블 SSD T7 Touch
3,한일시멘트,2021,레미탈(일반미장용)[40kg]
4,금호석유,2021,Hugreen 창호 프로파일
5,삼양식품,2023,사또밥[67g]
6,현대제철,2023,형강(H형강)
7,롯데칠성,2023,"칠성사이다 [1.5L,PET]"
8,아이에스동서,2023,"초고강도 PHC 파일 [110MPa, A종]"


#### ------------------------ Green 환경표지 --------------------------

In [ ]:
# 2024 환경표지 인증제품 현황(인증기간 유효, 날짜x)
excel_file_path = './green_datas/green_company.xlsx'
df_E_1 = pd.read_excel(excel_file_path)
df_E_1 = df_E_1.drop(columns=['번호', '전화번호', '기업주소' ])
df_E_1['인증연도'] = '2024'
df_E_1 = df_E_1.rename(columns={'업체명': '기업명'})
df_E_1 = df_E_1.rename(columns={'분류명': '제품명'})
df_E_1['기업명'] = df_E_1['기업명'].str.replace("\(주\)|주식회사|㈜|주\)|\(유\)|\)|에스아이씨엔티\n\(구.", "", regex=True)
df_E_1 = df_E_1[['기업명', '인증연도', '제품명']]
df_E_1

In [ ]:
# 2023 이전 환경표지 인증제품 현황 (제품 개수 o)
file_path = './green_datas/green_company_환경표지_2023.csv'
df_E_2 = pd.read_csv(file_path, encoding='cp949')
df_E_2 = df_E_2.drop(columns=['대상제품군',  '제품구분', '기본제품', '용도', '인증번호', '사업자번호', '본사지역구분', '공장지역구분', '공장구분', '인증사유','인증시작일', '인증종료일' ])
df_E_2['업체명'] = df_E_2['업체명'].str.replace("\(주\)|주식회사|㈜|주\)|\(유\)|\)|에스아이씨엔티\n\(구.", "", regex=True)
# df_4['인증일자'] = '2020-00-00'
df_E_2 = df_E_2.rename(columns={'업체명': '기업명'})

# # '기업명' 카운트 후 중복되는 '기업명' 제거
# counts = df_E_2['기업명'].value_counts()
# df_E_2['인증제품'] = df_E_2['기업명'].map(counts)
# df_E_2 = df_E_2.drop_duplicates(subset=['기업명']) 

# df_E_2 = df_E_2.drop(columns='인증제품')
df_E_2 = df_E_2.dropna(axis=0, how='all')

# 시작일 연도만 남겨놓기
df_E_2 = df_E_2.dropna()

df_E_2['최초인증일'] = pd.to_datetime(df_E_2['최초인증일'])
df_E_2['최초인증일'] = df_E_2['최초인증일'].dt.year.astype(int) 
df_E_2 = df_E_2.rename(columns={'최초인증일': '인증연도'})
df_E_2 = df_E_2[['기업명', '인증연도', '제품명']]
df_E_2

In [49]:
# 환경표지 인증 업체는 2023년까지의 데이터 + 2024년 추가
df_E = pd.concat([df_E_1, df_E_2])
df_E = df_E.drop_duplicates(subset=['기업명']) 

# '인증연도' 열을 숫자로 변환
df_E['인증연도'] = df_E['인증연도'].astype(int)

# '인증연도' 열을 기준으로 내림차순으로 정렬
df_E = df_E.sort_values(by='인증연도', ascending=True)
df_E = df_E.reset_index(drop=True)

df_E['기업명'] = df_E['기업명'].str.replace("\(DK 카페트|\(세림B&G|\(샛별재활원|\(사|\(MS스토어|\(C&B|\(소매유통본부|\(K&C|\(PAINT FAMILY|\(DS Water|\(WINDOW|\(합|\(복|\(이앤지|\(세림B&amp;amp;G|\(JJ라이팅|\(페르플러스|\(자|\(BG바이오|\(DASOM|\(THE WIN|\(ELCA-Furniture|\(Art one|\(합자|\(세제몰|\(C&amp;B co Ltd|\(K&amp;C,2024|\(TG Furniture|\(vanes|\(003205720200129532000160|\(SUZHOU CO. , LTD\"|\(실로암장애인근로사업장|\(001105720221209511002273|\(ECOPP|\(FIELD-LINE|\(WOOJIN|\(OBJET STUDIES|\(사단법인한국사회적일자리개발원|\(RS유통|\(ENCOM|\(태백장애인근로사업장|\(DAEJIN C&C|\(KNC|\(JS PACK_CHEMICAL|\(SGS|\(T.T.I|\(G.T.S코리아|\(한마음사업단|\(EPS|\(Artista chemical|\(HAP|\(IT. WORLD|\(Loofen Co., Ltd\"|\(춘천시장애인근로사업장|\(테스트|\(NCT|\(동방재활근로복지관|\(C·T|\(KNFUS|\(K&amp;C", "", regex=True)

# 기업명칭 통일
rename = {
    '효성ITX': '효성',
    'SK엔무브':'SK',
    '한샘넥서스': '한샘',
    '유한킴벌리': '유한양행',
    '금호석유화학': '금호석유', 
    '한화컴파운드': '한화',
    '위메이드 보호작업장':'위메이드',
}

for old_name, new_name in rename.items():
    df_E['기업명'] = df_E['기업명'].replace(old_name, new_name, regex=True)
    df_E['기업명'] = df_E['기업명'].str.strip()
rename_df_E = df_E
rename_df_E

,기업명,인증연도,제품명
0,한국하우톤,2001,Tectyl Hydro syn 46
1,대한민국상이군경회 I.T.S 사업소,2009,보훈전광판(KDVO-VMS-OF-1)
2,호진무역,2010,호진 수목장 분골함 大
3,경우단열,2010,경우몰 슈퍼-200
4,대전아스콘,2010,DJ-순환 가열 아스팔트 혼합물 기층용 BB-2
...,...,...,...
3933,유한양행,2024,[322] 방향제
3934,한성기업,2024,[746] 골재 및 미분말
3935,솔레즈,2024,[209] 일반조명용 LED 램프
3936,일흥건영,2024,[248] 벽 및 천장 마감재


In [27]:
# # rename_df_GR , top_200 두 데이터의 공통된 단어의 기업명
# words_df1 = set(' '.join(df_E ['기업명']).split())
# words_df2 = set(' '.join(top_200['기업명']).split())

# common_words = set()
# for word1 in words_df1:
#     for word2 in words_df2:
#         if word1 in word2 or word2 in word1:
#             common_words.add(word1)

# print("두 데이터프레임의 '기업명' 열에서 포함된 공통된 단어:", common_words)

두 데이터프레임의 '기업명' 열에서 포함된 공통된 단어: {'두산하이텍', '풍산마루', '효성오앤비', '세아베스틸', 'N', '미래', '한화', '효성첨단소재', '삼성', '한샘테크닉스', '금호석유', '한국콜마', '한국', '효성앤플러스', '대웅판넬', '로지', '강원랜드', '코웨이', '성화', '효성', '현대제철', '효성테크', '더', '세방전지', '에코', 'SK네트웍스', '에코웨이', '두산종합목재', '한샘', '효성프라콘', '현', '고려아연', '하나', '앙투카에스엘', '호텔신라', '(SGS', '오리온필름', '유풍산업', '효성산업개발', '아이', '피에스엘테크', '대상에스앤티', '한일', '효성에스피', '하이텍', '금호타이어', '삼성전자', '한국타이어앤테크놀로지', '동서개발', 'D', '효성조명', 'LS라이팅', 'SK', '코스', '(합자동서콘크리트', '씨앤에스원', '대웅시엔시', '신세계자원', '한풍산업', '에스엘이디조명', '이텍', '유한양행', '한일시멘트', '동서화학공업', '대인', '한국앤컴퍼니', '지에스엘바이오', '효성금속', '대한전선', '일진', '롯데케미칼', '앤'}


In [15]:
# 저탄소인증 기업리스트 중 top 200의 인증 기록
E_companies = ['효성',  'SK네트웍스', 'SK', '롯데케미칼', '한국콜마', '한국타이어앤테크놀로지', '한샘', '세방전지', '한일시멘트', '유한양행', '호텔신라',
                '강원랜드', '고려아연', '금호석유', '삼성전자', '코웨이', '효성첨단소재', '삼성전자', '한국앤컴퍼니', '금호타이어', '현대제철', '한화', '한샘', '대한전선']

top200_E = rename_df_E[rename_df_E['기업명'].isin(E_companies)]
top200_E = top200_E.reset_index(drop=True)
top200_E

,기업명,인증연도,제품명
0,한샘,2024,[223] 대변기
1,한국콜마,2024,[308] 샴푸·린스
2,한화,2024,[724] 생분해성 수지 제품
3,금호석유,2024,[243] 보온·단열재
4,금호타이어,2024,[502] 트럭·버스용 타이어
5,한국타이어앤테크놀로지,2024,[502] 트럭·버스용 타이어
6,롯데케미칼,2024,[727] 바이오매스 합성수지 제품
7,유한양행,2024,[303] 주방용 세제
8,세방전지,2024,[603] 산업용 축전지
9,한국앤컴퍼니,2024,[603] 산업용 축전지


#### --------------------------------- TOP 150 KOSDAQ ---------------------------------

In [20]:
# top_150 코스닥 기업리스트 불러오기 
file_path = 'top150_kosdaq.csv'
top_150 = pd.read_csv(file_path, sep='\t')
top_150 = top_150.rename(columns={'종목명': '기업명'})

#### ------------------------ Green GR --------------------------

In [22]:
# # rename_df_GR, top_150 두 데이터의 공통된 단어의 기업명
# words_df1 = set(' '.join(rename_df_GR['기업명']).split())
# words_df2 = set(' '.join(top_150['기업명']).split())

# common_words = set()
# for word1 in words_df1:
#     for word2 in words_df2:
#         if word1 in word2 or word2 in word1:
#             common_words.add(word1)

# print("두 데이터프레임의 '기업명' 열에서 포함된 공통된 단어:", common_words)

두 데이터프레임의 '기업명' 열에서 포함된 공통된 단어: {'이앤티', '덕산', '동화기업'}


In [23]:
# GR인증 기업리스트 중 top 150의 인증 기록
GR_companies = ['동화기업']

top_150_GR = rename_df_GR[rename_df_GR['기업명'].isin(GR_companies)]
top_150_GR = top_150_GR.reset_index(drop=True)
top_150_GR

,기업명,인증연도,분야
0,동화기업,2011,폐목재


#### ------------------------ 저탄소 인증 --------------------------

In [25]:
# # rename_df_C, top_150 두 데이터의 공통된 단어의 기업명
# words_df1 = set(' '.join(rename_df_C['기업명']).split())
# words_df2 = set(' '.join(top_150['기업명']).split())

# common_words = set()
# for word1 in words_df1:
#     for word2 in words_df2:
#         if word1 in word2 or word2 in word1:
#             common_words.add(word1)

# print("두 데이터프레임의 '기업명' 열에서 포함된 공통된 단어:", common_words)

두 데이터프레임의 '기업명' 열에서 포함된 공통된 단어: {'강원', '동화기업'}


In [26]:
# 저탄소인증 기업리스트 중 top 150의 인증 기록
C_companies = ['동화기업']

top_150_C = rename_df_C[rename_df_C['기업명'].isin(C_companies)]
top_150_C = top_150_C.reset_index(drop=True)
top_150_C

3,기업명,인증연도,제품명
0,동화기업,2020,나투스 진 [98mmx815mmx7.5mm]


#### ------------------------ 환경표지 인증 --------------------------

In [28]:
# # rename_df_E, top_150 두 데이터의 공통된 단어의 기업명
# words_df1 = set(' '.join(rename_df_E['기업명']).split())
# words_df2 = set(' '.join(top_150['기업명']).split())

# common_words = set()
# for word1 in words_df1:
#     for word2 in words_df2:
#         if word1 in word2 or word2 in word1:
#             common_words.add(word1)

# print("두 데이터프레임의 '기업명' 열에서 포함된 공통된 단어:", common_words)

두 데이터프레임의 '기업명' 열에서 포함된 공통된 단어: {'앤', '위메이드', '한국', '오스코', '현', '천보콘크리트', '티엠', 'N', '아이티', '화이어프로텍', '셀바이오', '한국환경산업기술원(테스트', '엘에스엠앤엠', '서진', '이텍', '하나', '이엠', '아이', '에스엠산업개발', '에스엠소재', '에스엠마루', '케이', '젬', 'D', '하이텍', '천보건설', '다원', '티엠로직스(세제몰', '에스엠티', '동화기업', '미래', 'HI', '에코', '에스에스엠엠', '더', '이노바텍', '에프에스티테크놀로지'}


In [32]:
# 저탄소인증 기업리스트 중 top 150의 인증 기록
C_companies = ['동화기업', '위메이드']

top_150_E = rename_df_E[rename_df_E['기업명'].isin(C_companies)]
top_150_E = top_150_E.reset_index(drop=True)
top_150_E

,기업명,인증연도,제품명
0,동화기업,2024,[248] 벽 및 천장 마감재
1,위메이드,2024,[147] 컴퓨터용 모니터


In [44]:
# top_150_E.to_csv('top150_green_E_list.csv', index=False)